In [1]:
import boto3, re
import sagemaker
from sagemaker import get_execution_role
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
import time
from sagemaker.predictor import csv_serializer, json_deserializer,RealTimePredictor

sagemaker_session = sagemaker.Session()
role = get_execution_role()
bucket='tensorflow-rul'
prefix='Keras'

In [2]:
train_set = pd.read_csv('newtrain.csv', index_col = 0)
test_set = pd.read_csv('newtest.csv', index_col = 0)
train=pd.DataFrame(train_set)
test=pd.DataFrame(test_set)

In [3]:
train_x=train.drop(columns='RUL')
train_y=train['RUL']
test_x=test.drop(columns='RUL')
test_y=test['RUL']
train_x=np.array(train_x)
train_y=np.array(train_y)
test_x=np.array(test_x)
test_y=np.array(test_y)


In [4]:
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))
# train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
# test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))

In [5]:
os.makedirs("./data", exist_ok = True)
np.savez('./data/training', image=train_x, label=train_y)
np.savez('./data/validation', image=test_x, label=test_y)

In [6]:
# training_input_path   = sagemaker_session.upload_data('data/training' ,bucket , key_prefix=prefix+'/training')
# validation_input_path = sagemaker_session.upload_data('data/validation' ,bucket , key_prefix=prefix+'/validation')
training_input_path = sagemaker_session.upload_data(path='data/training.npz',bucket=bucket, key_prefix=prefix+'/training')
validation_input_path = sagemaker_session.upload_data('data/validation.npz' ,bucket=bucket , key_prefix=prefix+'/validation')


In [7]:
from sagemaker.tensorflow import TensorFlow
output_path = "s3://tensorflow-rul/Keras/output_model"
tf_estimator = TensorFlow(entry_point='lstm_keras.py', 
                          role=role,
                          output_path = output_path,
                          train_instance_count=1, 
                          train_instance_type='ml.c4.xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 10,
                              'batch-size': 512,
                              'learning-rate': 0.01}
                         )

In [8]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-16 09:43:48 Starting - Starting the training job...
2020-10-16 09:43:50 Starting - Launching requested ML instances.........
2020-10-16 09:45:21 Starting - Preparing the instances for training...
2020-10-16 09:46:03 Downloading - Downloading input data...
2020-10-16 09:46:37 Training - Training image download completed. Training in progress.2020-10-16 09:46:40,811 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-10-16 09:46:40,817 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-16 09:46:41,109 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-16 09:46:41,126 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-10-16 09:46:41,139 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training",
        "validatio

 - 9s - loss: 4744.5831 - mean_absolute_percentage_error: 522145553.6072 - val_loss: 4600.8087 - val_mean_absolute_percentage_error: 49.3824
Epoch 8/10
 - 9s - loss: 4744.6642 - mean_absolute_percentage_error: 522922717.3708 - val_loss: 4583.0339 - val_mean_absolute_percentage_error: 49.3959
Epoch 9/10
 - 9s - loss: 4744.6747 - mean_absolute_percentage_error: 521217952.5615 - val_loss: 4601.0823 - val_mean_absolute_percentage_error: 49.3822
Epoch 10/10

2020-10-16 09:48:19 Uploading - Uploading generated training model - 9s - loss: 4744.5858 - mean_absolute_percentage_error: 523361143.6583 - val_loss: 4592.0996 - val_mean_absolute_percentage_error: 49.3890
#015   32/13096 [..............................] - ETA: 4s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  224/13096 [..............................] - ETA: 3s#01


2020-10-16 09:48:26 Completed - Training job completed
Training seconds: 143
Billable seconds: 143


In [10]:
output_path = '/'.join(tf_estimator.output_path.split('/')[:-1])

print(output_path)
# 

optimized_estimator = tf_estimator.compile_model(target_instance_family='rasp3b',
                              output_path=output_path,
                              input_shape= {'data':[26,1]},  # Batch size 1, 3 channels, 224x224 Images.
                              framework='keras', framework_version='1.12')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


s3://tensorflow-rul/Keras
?....*

UnexpectedStatusException: Error for Compilation job compilation-sagemaker-tensorflow-script-2020-10-16-09-49-27-383: Failed. Reason: ServerError: Internal server error during compilation. Please contact amazon-neo-feedback@amazon.com with your Job ARN to get support for this compilation job.